# Chạy 3 khối này trước

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import random

import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array

import warnings
warnings.filterwarnings('ignore')

In [3]:
class Config:
    """ Cài đặt mô hình và huấn luyện """
     # Đường dẫn dữ liệu
    data_dir = ("/content/drive/MyDrive/Dataset/Images")
    # Cài đặt hình ảnh
    IMG_HEIGHT = 48
    IMG_WIDTH = 48
    IMG_CHANNELS = 1

    # Tham số huấn luyện
    BATCH_SIZE=32
    EPOCHS=50
    LEARNING_RATE=0.0001

    # Các lớp cảm xúc
    CLASSES=['tức giận', 'ghê tởm',"sợ hãi","hạnh phúc","trung tính","buồn","ngạc nhiên"]
    NUM_CLASSES=len(CLASSES)

    # Đường dẫn lưu trữ
    MODEL_SAVE_PATH="fer_2013_model.h5"
    WEIGHTS_SAVE_PATH="fer_2013_weights.h5"
    HISTORY_SAVE_PATH="training_history.npy"

config=Config()

# Kéo xuống chạy phần thử nghiệm

In [49]:
# ==============================
# 3. Chuẩn bị dữ liệu với Tăng cường dữ liệu (Data Augmentation)
# ==============================
def create_data_generators():
    TRAIN_DIR = os.path.join(config.data_dir, 'train')

    TEST_DIR = os.path.join(config.data_dir ,"test")

    # Bộ sinh dữ liệu huấn luyện với nhiều tối ưu hóa
    train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 10,          # Xoay ảnh ngẫu nhiên trong khoảng 10 độ
        width_shift_range = 0.1,      # Dịch ngang ảnh
        height_shift_range = 0.1,     # Dịch dọc ảnh
        shear_range = 0.1,            # Biến dạng cắt (shear)
        horizontal_flip = True,       # Lật ngang ảnh
        fill_mode  = "nearest",       # Chế độ điền pixel khi biến đổi
        validation_split = 0.2)       # Tách 20% dữ liệu để xác thực

    # Bộ sinh dữ liệu xác thực & kiểm tra (chỉ chuẩn hóa lại giá trị pixel)
    test_datagen  = ImageDataGenerator(rescale=1./255)

    # Bộ sinh dữ liệu huấn luyện
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size = (config.IMG_HEIGHT, config.IMG_WIDTH),
        color_mode  = "grayscale",    # Ảnh thang độ xám
        batch_size  = config.BATCH_SIZE,
        class_mode  = "categorical",  # Phân loại nhiều lớp
        shuffle     = True,
        seed        = 42,             # Đảm bảo tính ổn định giữa các lần chạy
        subset="training"
    )

    # Bộ sinh dữ liệu xác thực (từ cùng thư mục huấn luyện, dùng validation_split)
    validation_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size = (config.IMG_HEIGHT, config.IMG_WIDTH),
        color_mode = "grayscale",
        batch_size = config.BATCH_SIZE,
        class_mode = "categorical",
        shuffle    = False,
        seed       = 42,              # Đảm bảo tính ổn định giữa các lần chạy
        subset="validation"
    )

    # Bộ sinh dữ liệu kiểm tra
    test_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size = (config.IMG_HEIGHT, config.IMG_WIDTH),
        color_mode = "grayscale",
        batch_size = config.BATCH_SIZE,
        class_mode = "categorical",
        shuffle    = False,
    )

    return train_generator, validation_generator, test_generator

In [50]:
# ==============================
# 4. Xây dựng mô hình CNN chuyên nghiệp
# ==============================
def build_cnn_model():
    model = models.Sequential([
        # Khối 1 - Trích xuất đặc trưng
        layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same",
                      input_shape=(config.IMG_HEIGHT, config.IMG_WIDTH, config.IMG_CHANNELS)
                     ),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),  # Giảm kích thước không gian (spatial dimension)
        layers.Dropout(0.25),

        # Khối 2
        layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same"),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.25),

        # Khối 3
        layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same"),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation="relu", kernel_initializer='he_normal', padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.30),

        # Flatten + Dense - Bộ phân loại
        layers.Flatten(),

        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.40),

        # Lớp đầu ra
        layers.Dense(config.NUM_CLASSES, activation="softmax")
    ])
    return model

In [51]:
# ==============================
# 5. Biên dịch (Assemble) mô hình
# =================================
def compile_model(model):
     optimizer = optimizers.Adam(learning_rate=config.LEARNING_RATE)
     model.compile(
         optimizer = optimizer,
         loss = "categorical_crossentropy",   # Hàm mất mát cho phân loại đa lớp
         metrics = [
             "accuracy",                      # Độ chính xác
             tf.keras.metrics.Precision(),    # Độ chính xác (Precision)
             tf.keras.metrics.Recall(),       # Độ thu hồi (Recall)
             tf.keras.metrics.AUC()           # Diện tích dưới đường cong ROC (AUC)
         ]
     )
     return model

In [52]:
# ==============================
# 6. Callbacks (Hàm gọi lại trong quá trình huấn luyện)
# ==============================

def create_callbacks():
    return [
        # Lưu mô hình có độ chính xác xác thực (val_accuracy) tốt nhất
        callbacks.ModelCheckpoint(config.MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),

        # Giảm learning rate nếu val_loss không cải thiện sau 5 epoch
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),

        # Dừng huấn luyện sớm nếu val_loss không cải thiện sau 10 epoch
        # và khôi phục lại trọng số tốt nhất
        callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),

        # Lưu lại lịch sử huấn luyện vào file CSV
        callbacks.CSVLogger(config.HISTORY_SAVE_PATH),
    ]

In [53]:
# ==============================
# 7. Huấn luyện mô hình
# ==============================

def train_model(model, train_gen, val_gen, cb_list):
    print("\n🚀 Bắt đầu huấn luyện...\n")
    history = model.fit(
        train_gen,
        epochs=config.EPOCHS,
        validation_data=val_gen,
        callbacks=cb_list,
        verbose=1
    )
    np.save(config.HISTORY_SAVE_PATH, history.history)
    print(f"\n✅ Huấn luyện hoàn tất! Độ chính xác xác thực tốt nhất: {max(history.history['val_accuracy']):.4f}")

    # ==============================
    # Vẽ đường cong Huấn luyện và Độ chính xác
    # ==============================

    # Độ chính xác
    plt.figure(figsize=(8,5))
    plt.plot(history.history['accuracy'], label='Độ chính xác huấn luyện')
    plt.plot(history.history['val_accuracy'], label='Độ chính xác xác thực')
    plt.title('Độ chính xác của mô hình')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Hàm mất mát
    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'], label='Mất mát huấn luyện')
    plt.plot(history.history['val_loss'], label='Mất mát xác thực')
    plt.title('Hàm mất mát của mô hình')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    return history

In [54]:
def predict_and_analyze(model, test_gen):
    print("\n🚀 Dự đoán trên dữ liệu kiểm tra...\n")

    # Dự đoán phân loại
    y_pred_prob = model.predict(test_gen)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = test_gen.classes

    # Báo cáo phân loại
    print("\n📄 Báo cáo phân loại:")
    print(classification_report(y_true, y_pred, target_names=config.CLASSES))

    # Ma trận nhầm lẫn
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=config.CLASSES,
                yticklabels=config.CLASSES)
    plt.title('📊 Ma trận nhầm lẫn')
    plt.xlabel("Dự đoán")
    plt.ylabel('Thực tế')
    plt.show()
    plt.tight_layout()

In [55]:
def evaluate_model(model, test_gen):
    print("\n✅ Đang đánh giá mô hình...\n")

    results = model.evaluate(test_gen, verbose=1)

    # Liên kết kết quả với tên các chỉ số
    metrics_names = model.metrics_names  # ['loss', 'accuracy', 'precision', 'recall', 'auc']
    results_dict = dict(zip(metrics_names, results))

    print("\n📊 Kết quả kiểm tra:")
    for name, value in results_dict.items():
        print(f"{name.capitalize()}: {value:.4f}")

    return results_dict.get('accuracy', None)

In [56]:
# ==============================
# Hàm chính cho quá trình huấn luyện và đánh giá CNN
# ==============================
def main():
    print("\n" + "="*60)
    print(" Mô hình CNN cho Nhận diện cảm xúc khuôn mặt ")
    print("="*60 + "\n")

    # ------------------------------
    # Chuẩn bị dữ liệu
    # ------------------------------
    # train_gen, val_gen, test_gen là các Keras ImageDataGenerators
    train_gen, val_gen, test_gen = create_data_generators()

    # ------------------------------
    # Xây dựng và biên dịch mô hình CNN
    # ------------------------------
    # build_cnn_model() trả về kiến trúc mô hình
    # compile_model() biên dịch mô hình với optimizer, loss, và metrics
    model = compile_model(build_cnn_model())

    # ------------------------------
    # Huấn luyện mô hình
    # ------------------------------
    # train_model() huấn luyện mô hình trên dữ liệu train và xác thực trên val_gen
    history = train_model(model, train_gen, val_gen, create_callbacks())

    # ------------------------------
    # Đánh giá mô hình
    # ------------------------------
    # Đánh giá trên tập test
    evaluate_model(model, test_gen)

    # ------------------------------
    # Dự đoán và phân tích kết quả
    # ------------------------------
    predict_and_analyze(model, test_gen)

    # ------------------------------
    # Lưu mô hình đã huấn luyện
    # ------------------------------
    model.save(config.MODEL_SAVE_PATH)
    print(f"\n✅ Mô hình đã được lưu tại: {config.MODEL_SAVE_PATH}")

In [ ]:

# ==============================
# 11. Run program
# ==============================
if __name__ == "__main__":
    # Run the main training and evaluation pipeline
    main()

    # Example: single image prediction (update the path to your image)
    #predict_single_image(
        #model_path=config.MODEL_SAVE_PATH,
        #image_path='path/to/your/image.jpg'  # <- replace with your actual image path
    #)



 Mô hình CNN cho Nhận diện cảm xúc khuôn mặt 

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.

🚀 Bắt đầu huấn luyện...

Epoch 1/50
  5/718 ━━━━━━━━━━━━━━━━━━━━ 1:25:56 7s/step - accuracy: 0.1679 - auc: 0.4779 - loss: 3.1178 - precision: 0.1412 - recall: 0.0643

In [ ]:
# 2. Chuẩn bị dữ liệu kiểm tra
# Chúng ta chỉ cần test_generator để đánh giá và dự đoán
_, _, test_gen = create_data_generators()
print("✅ Bộ sinh dữ liệu kiểm tra đã được tạo.")

# 3. Đánh giá mô hình đã tải trên tập kiểm tra
print("\n--- Đang đánh giá mô hình đã tải ---")
evaluate_model(loaded_model, test_gen)

# 4. Thực hiện dự đoán và phân tích kết quả với mô hình đã tải
print("\n--- Thực hiện dự đoán với mô hình đã tải ---")
predict_and_analyze(loaded_model, test_gen)

# **Thử nghiệm ở đây **
block đầu để load model

In [ ]:

import tensorflow as tf
from tensorflow.keras import models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the Config class locally to ensure 'config' is available
class Config:
    """ Model and training settings """
    #Data direction
    data_dir = ("/content/drive/MyDrive/Dataset/Images") # Adjusted path for dataset
    # Image transactions
    IMG_HEIGHT = 48
    IMG_WIDTH = 48
    IMG_CHANNELS = 1

    # TRAINING parameters
    BATCH_SIZE=32
    EPOCHS=50
    LEARNING_RATE=0.0001

    #Categories
    CLASSES=['angry', 'disgust',"fear","happy","neutral","sad","surprise"]
    NUM_CLASSES=len(CLASSES)

    # Saving paths
    MODEL_SAVE_PATH="/content/drive/MyDrive/Deep_learning/fer_2013_model.h5"
    WEIGHTS_SAVE_PATH="fer_2013_weights.h5"
    HISTORY_SAVE_PATH="training_history.npy"

config=Config()

# ============================== # 3. Data preparation with Data Augmentation # ==============================
def create_data_generators():
    actual_data_dir = config.data_dir # Default from Config class
    # The previous cell downloaded to '/content/datasets' and unzipped it.
    # So, the actual path might be '/content/datasets/fer2013' if 'fer2013' is the root folder inside the zip.
    # Let's adjust data_dir in config if it's not pointing correctly based on download.
    # For this specific run, let's make sure it looks for the correct path.
    # The previous code had a check, but now config.data_dir is directly set to the correct path.

    TRAIN_DIR = os.path.join(actual_data_dir, 'train')
    TEST_DIR = os.path.join(actual_data_dir ,"test")

    # Check if directories exist before proceeding
    if not os.path.exists(TRAIN_DIR) or not os.path.exists(TEST_DIR):
        print(f"Error: Training or Test directory not found. Please ensure dataset is correctly extracted at {actual_data_dir}.")
        # Return empty generators or raise an error
        raise FileNotFoundError(f"Required data directories not found at {TRAIN_DIR} and {TEST_DIR}")

    # Training data generator with multiple optimizations
    train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        shear_range = 0.1,
        horizontal_flip = True,
        fill_mode  = "nearest",
        validation_split = 0.2)

    # Validation & Test generator (just rescale)
    test_datagen  = ImageDataGenerator(rescale=1./255)

    # Training generator
    train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (config.IMG_HEIGHT,config.IMG_WIDTH),
    color_mode   = "grayscale",
    batch_size  = config.BATCH_SIZE,
    class_mode  = "categorical",
    shuffle      = True ,
    seed        = 42, #To ensure sample stability between sessions.
    subset="training"
    )

    # Validation generator (from same training dir using validation_split)
    validation_generator=train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (config.IMG_HEIGHT,config.IMG_WIDTH),
    color_mode = "grayscale",
    batch_size = config.BATCH_SIZE,class_mode="categorical",
    shuffle    = False,
    seed       = 42 ,      #To ensure sample stability between sessions.
     subset="validation"
     )
    test_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size = (config.IMG_HEIGHT,config.IMG_WIDTH),
        color_mode = "grayscale",
        batch_size = config.BATCH_SIZE,
        class_mode = "categorical",
        shuffle     = False
            )

    return train_generator ,validation_generator,test_generator

def evaluate_model(model, test_gen):
    print("\n✅ Evaluating model...\n")

    results = model.evaluate(test_gen, verbose=1)

    # Associate results with metrics names
    metrics_names = model.metrics_names  # ['loss', 'accuracy', 'precision', 'recall', 'auc']
    results_dict = dict(zip(metrics_names, results))

    print("\n📊 Test Results:")
    for name, value in results_dict.items():
        print(f"{name.capitalize()}: {value:.4f}")

    return results_dict.get('accuracy', None)

def predict_and_analyze(model, test_gen):
    print("\n🚀 Prediction on test data...\n")

   #Category prediction
    y_pred_prob = model.predict(test_gen)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = test_gen.classes

    # Classification Report
    print("\n📄 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=config.CLASSES))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=config.CLASSES,
                yticklabels=config.CLASSES)
    plt.title('📊 Confusion Matrix')
    plt.xlabel("Predicted")
    plt.ylabel('Actual')
    plt.show()
    plt.tight_layout()

print("\n" + "="*60)
print(" Using Saved Model for Facial Emotion Recognition Testing ")
print("="*60 + "\n")

# 1. Load the pre-trained model
try:
    loaded_model = models.load_model(config.MODEL_SAVE_PATH)
    print(f"✅ Model '{config.MODEL_SAVE_PATH}' loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model '{config.MODEL_SAVE_PATH}': {e}")
    print("Please ensure the model file exists and the path is correct.")
    exit()

# Upload ảnh để kiểm tra

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import io

# Assuming 'config' object and 'loaded_model' are already defined and accessible from previous cells.
# If not, you may need to run the cell where they are defined first.

def predict_single_image_from_upload(model, config):
    print("\n--- Upload Image for Prediction ---")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return

    # Get the name of the uploaded file
    for fn in uploaded.keys():
        print(f'User uploaded file "{fn}"')
        image_path = io.BytesIO(uploaded[fn])

        try:
            # Load and preprocess the image
            img = load_img(image_path, color_mode='grayscale', target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
            img_array = img_array / 255.0 # Rescale to 0-1

            # Make prediction
            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions[0])
            predicted_emotion = config.CLASSES[predicted_class_index]
            confidence = predictions[0][predicted_class_index] * 100

            # Display the image and prediction
            plt.figure(figsize=(4, 4))
            plt.imshow(img, cmap='gray')
            plt.title(f"Predicted Emotion: {predicted_emotion} ({confidence:.2f}%)")
            plt.axis('off')
            plt.show()

        except Exception as e:
            print(f"Error processing image {fn}: {e}")

# Call the function to test
# Ensure 'loaded_model' and 'config' are available from earlier execution.
# If you run this cell independently, make sure the cell defining loaded_model and config has been run.
if 'loaded_model' in locals() and 'config' in locals():
    predict_single_image_from_upload(loaded_model, config)
else:
    print("Error: 'loaded_model' or 'config' not found. Please run the previous cells first to load the model and define config.")


# Lấy 10 ảnh từ tập test để thử nghiệm

## Phân tích quy trình huấn luyện và phân loại cảm xúc

Dựa trên các ô code đã cung cấp, đây là cách mô hình được huấn luyện và cách nó phân loại cảm xúc:

**1. Tổng quan quy trình:**

Quy trình diễn ra theo các bước chính sau:
*   **Cấu hình (Config):** Định nghĩa các tham số chung cho mô hình và quá trình huấn luyện.
*   **Chuẩn bị dữ liệu (Data Preparation):** Tải dữ liệu, tiền xử lý và áp dụng tăng cường dữ liệu (data augmentation).
*   **Xây dựng mô hình (Model Building):** Định nghĩa kiến trúc mạng CNN.
*   **Biên dịch mô hình (Compile Model):** Cấu hình thuật toán tối ưu hóa, hàm mất mát và các chỉ số đánh giá.
*   **Huấn luyện mô hình (Model Training):** Đào tạo mô hình trên dữ liệu huấn luyện và xác thực.
*   **Đánh giá & Phân tích (Evaluation & Analysis):** Kiểm tra hiệu suất mô hình trên dữ liệu kiểm tra và phân tích kết quả.
*   **Dự đoán ảnh đơn (Single Image Prediction):** Sử dụng mô hình đã huấn luyện để dự đoán cảm xúc từ một ảnh mới.

**2. Chi tiết từng bước:**

*   **`Config` (Cell E1-GduF7y0Sf và z5QFO2F-1dOF):**
    *   Lớp `Config` chứa tất cả các cài đặt quan trọng như: đường dẫn dữ liệu (`data_dir`), kích thước ảnh (`IMG_HEIGHT`, `IMG_WIDTH`, `IMG_CHANNELS`), kích thước batch (`BATCH_SIZE`), số epoch (`EPOCHS`), tốc độ học (`LEARNING_RATE`), các lớp cảm xúc (`CLASSES`) và đường dẫn lưu mô hình/trọng số.
    *   Các giá trị này sẽ được sử dụng xuyên suốt quá trình huấn luyện và đánh giá.

*   **`create_data_generators()` (Cell 3XG1SnRlzGRP và z5QFO2F-1dOF):**
    *   Hàm này sử dụng `ImageDataGenerator` của Keras để tải và tiền xử lý dữ liệu ảnh.
    *   **Tăng cường dữ liệu (Data Augmentation):** Đối với tập huấn luyện, `ImageDataGenerator` áp dụng các phép biến đổi ngẫu nhiên như xoay (rotation), dịch chuyển (shift), cắt xén (shear), lật ngang (horizontal flip) để tạo ra các biến thể của ảnh gốc. Điều này giúp mô hình học cách tổng quát hóa tốt hơn và giảm overfitting.
    *   **Tiền xử lý:** Rescale giá trị pixel từ 0-255 về 0-1.
    *   **Tạo Generators:** Tạo ra các `train_generator`, `validation_generator` và `test_generator` giúp load dữ liệu theo từng batch hiệu quả, đặc biệt với các tập dữ liệu lớn không thể chứa hết trong bộ nhớ.

*   **`build_cnn_model()` (Cell lrwaRl43zGyn):**
    *   Hàm này định nghĩa kiến trúc của Mạng nơ-ron tích chập (CNN) sẽ được sử dụng.
    *   Mô hình là một chuỗi (`Sequential`) các lớp:
        *   **`Conv2D`:** Các lớp tích chập để trích xuất đặc trưng từ ảnh.
        *   **`BatchNormalization`:** Giúp ổn định quá trình huấn luyện bằng cách chuẩn hóa các đầu ra của lớp trước đó.
        *   **`MaxPooling2D`:** Giảm kích thước không gian của các bản đồ đặc trưng, giúp giảm số lượng tham số và tăng tốc độ tính toán.
        *   **`Dropout`:** Ngẫu nhiên bỏ qua một số nơ-ron trong quá trình huấn luyện để ngăn ngừa overfitting.
        *   **`Flatten`:** Chuyển đổi đầu ra từ các lớp tích chập thành một vector 1D.
        *   **`Dense`:** Các lớp fully connected (kết nối đầy đủ) để học các mối quan hệ phức tạp và đưa ra quyết định phân loại.
        *   **`softmax` (Output Layer):** Lớp đầu ra với hàm kích hoạt softmax, cho ra xác suất thuộc về từng lớp cảm xúc.

*   **`compile_model()` (Cell 8AeL403BzMaf):**
    *   Biên dịch mô hình bằng cách xác định:
        *   **Optimizer (Trình tối ưu hóa):** `Adam` được sử dụng với `learning_rate` xác định trong `Config`. Trình tối ưu hóa điều chỉnh trọng số của mô hình để giảm thiểu hàm mất mát.
        *   **Loss Function (Hàm mất mát):** `categorical_crossentropy` được sử dụng cho bài toán phân loại đa lớp (multi-class classification). Hàm này đo lường sự khác biệt giữa phân phối xác suất dự đoán và phân phối xác suất thực tế.
        *   **Metrics (Chỉ số đánh giá):** `accuracy` (độ chính xác), `Precision` (độ chính xác), `Recall` (độ thu hồi) và `AUC` (diện tích dưới đường cong ROC) được theo dõi trong quá trình huấn luyện để đánh giá hiệu suất của mô hình.

*   **`create_callbacks()` (Cell vCCfQCXzzPHO):**
    *   Các callback là các hàm được gọi tại các điểm khác nhau trong quá trình huấn luyện để thực hiện các tác vụ như:
        *   **`ModelCheckpoint`:** Lưu mô hình có `val_accuracy` tốt nhất.
        *   **`ReduceLROnPlateau`:** Giảm `learning_rate` nếu `val_loss` không cải thiện sau một số epoch nhất định.
        *   **`EarlyStopping`:** Dừng huấn luyện sớm nếu `val_loss` không cải thiện, để ngăn chặn overfitting và khôi phục lại trọng số của mô hình tốt nhất.
        *   **`CSVLogger`:** Lưu lại lịch sử huấn luyện vào một file CSV.

*   **`train_model()` (Cell mFVvm0VBzPyf):**
    *   Hàm này thực hiện quá trình huấn luyện mô hình bằng cách gọi `model.fit()`.
    *   Mô hình được huấn luyện trên `train_generator`, xác thực trên `validation_generator` và sử dụng các `callbacks` đã định nghĩa.
    *   Lịch sử huấn luyện (loss, accuracy, v.v. cho cả tập train và validation) được lưu lại và sau đó vẽ biểu đồ để dễ dàng theo dõi sự thay đổi qua các epoch.

*   **`evaluate_model()` (Cell CG9oRYOWzUDP và z5QFO2F-1dOF):**
    *   Sau khi huấn luyện, hàm này đánh giá hiệu suất cuối cùng của mô hình trên tập `test_generator` (dữ liệu kiểm tra mà mô hình chưa từng thấy).
    *   Nó in ra các chỉ số như loss, accuracy, precision, recall, và AUC trên tập kiểm tra.

*   **`predict_and_analyze()` (Cell GMPzudkFzRc3 và z5QFO2F-1dOF):**
    *   Hàm này thực hiện dự đoán trên tập `test_generator`.
    *   **`classification_report`:** Hiển thị chi tiết các chỉ số precision, recall, f1-score và support cho từng lớp cảm xúc.
    *   **`confusion_matrix`:** Vẽ biểu đồ ma trận nhầm lẫn, cho thấy số lượng mẫu được phân loại đúng/sai giữa các lớp, giúp bạn hiểu rõ mô hình nhầm lẫn giữa các cảm xúc nào.

*   **`predict_single_image_from_upload()` (Cell OLB-1snH4IPM):**
    *   Chức năng này cho phép bạn tải lên một hình ảnh từ máy tính của mình.
    *   Ảnh được tải lên sẽ được tiền xử lý (chuyển sang thang độ xám, thay đổi kích thước, chuẩn hóa) để phù hợp với định dạng đầu vào của mô hình.
    *   Mô hình `loaded_model` sau đó sẽ đưa ra dự đoán về cảm xúc trong ảnh.
    *   Kết quả dự đoán (cảm xúc và độ tin cậy) cùng với hình ảnh sẽ được hiển thị.

**Tóm lại:**

Mô hình được huấn luyện bằng cách sử dụng mạng CNN để học các đặc trưng từ ảnh khuôn mặt. Quá trình này được hỗ trợ bởi tăng cường dữ liệu và các kỹ thuật điều chỉnh như Batch Normalization và Dropout để cải thiện hiệu suất. Sau khi huấn luyện, mô hình có thể phân loại cảm xúc bằng cách xử lý ảnh đầu vào và đưa ra dự đoán xác suất cho từng loại cảm xúc.

Tuyệt vời! Tôi đã tạo một kế hoạch để phân tích nội dung file, tập trung vào các mô hình, thư viện và xu hướng hiện tại trong nhận dạng cảm xúc khuôn mặt theo yêu cầu của bạn. Bạn hãy xem xét và cho tôi biết nếu có bất kỳ điều chỉnh nào nhé.

Analyze Model Architecture: Phân tích và giải thích kiến trúc mạng CNN được định nghĩa trong hàm build_cnn_model (Cell lrwaRl43zGyn), tập trung vào các lớp của nó (Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense) và cách chúng đóng góp vào việc trích xuất đặc trưng và phân loại.
Identify Deep Learning Framework and Libraries: Liệt kê framework học sâu (TensorFlow/Keras) và các thư viện Python chính (numpy, matplotlib, seaborn, sklearn) được sử dụng để xây dựng mô hình, huấn luyện, đánh giá và trực quan hóa trong notebook.
Discuss Data Preparation and Augmentation: Giải thích cách ImageDataGenerator được sử dụng để tải dữ liệu, tiền xử lý (rescaling) và các kỹ thuật tăng cường dữ liệu (rotation, shifts, shear, horizontal flip) trong create_data_generators (Cell 3XG1SnRlzGRP).
Compare with Traditional Models: Thảo luận ngắn gọn về các phương pháp học máy truyền thống như SVM, KNN và LBP trong bối cảnh nhận dạng cảm xúc khuôn mặt, làm nổi bật sự khác biệt của chúng so với CNN học sâu được triển khai trong notebook (ví dụ: kỹ thuật feature engineering so với học đặc trưng tự động).
Elaborate on Deep Learning Architectures: Mở rộng về kiến trúc CNN được sử dụng và giới thiệu ngắn gọn các mô hình học sâu tiên tiến hơn như ResNet và Inception, những mô hình thường được sử dụng cho các tác vụ nhận dạng hình ảnh, lưu ý những lợi ích tiềm năng của chúng cho nhận dạng cảm xúc khuôn mặt.
Explain Supporting Libraries: Mô tả vai trò của Keras/TensorFlow trong triển khai hiện tại. Đồng thời, giải thích cách các thư viện như OpenCV (để xử lý video thời gian thực, phát hiện khuôn mặt) và DeepFace (để sử dụng các mô hình đã được huấn luyện trước, API phân tích khuôn mặt) có thể được tích hợp hoặc có liên quan trong một hệ thống nhận dạng cảm xúc khuôn mặt hoàn chỉnh, ngay cả khi chúng không được sử dụng trực tiếp trong script huấn luyện được cung cấp.
Summarize Current Trends: Kết luận bằng cách tóm tắt bối cảnh hiện tại của nhận dạng cảm xúc khuôn mặt, nhấn mạnh sự thống trị của các phương pháp học sâu và tầm quan trọng của tăng cường dữ liệu, kiến trúc mô hình mạnh mẽ và các chỉ số hiệu suất.
Final Task: Tổng hợp tất cả các phát hiện vào một phân tích toàn diện về các mô hình, thư viện và xu hướng trong nhận dạng cảm xúc khuôn mặt dựa trên nội dung của notebook và yêu cầu của bạn.

tôi muốn bạn hãy viết nhưng thư viện, kiếntrucs, mô hình được sử dụng trong file này
Rất tiếc, tôi đã gặp lỗi. Bạn có thể thử lại không?
Dựa trên các ô code bạn đã cung cấp, đây là tổng hợp các thư viện, kiến trúc và mô hình được sử dụng:

1. Thư viện được sử dụng:

google.colab.drive: Để kết nối với Google Drive.
os: Để tương tác với hệ điều hành (quản lý đường dẫn, thư mục).
random: Để tạo số ngẫu nhiên (sử dụng cho seed trong ImageDataGenerator).
numpy (as np): Để xử lý mảng số học.
matplotlib.pyplot (as plt): Để tạo biểu đồ và trực quan hóa dữ liệu.
seaborn (as sns): Để tạo biểu đồ thống kê chuyên nghiệp (ví dụ: ma trận nhầm lẫn).
sklearn.metrics: Để đánh giá hiệu suất mô hình (classification_report, confusion_matrix).
tensorflow (as tf): Framework học sâu chính.
tensorflow.keras: Các module để xây dựng và huấn luyện mô hình học sâu:
layers: Để định nghĩa các lớp của mạng neural (Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense).
models: Để tạo mô hình (Sequential) và tải/lưu mô hình (load_model, save).
optimizers: Để định nghĩa trình tối ưu hóa (Adam).
callbacks: Để sử dụng các callback trong quá trình huấn luyện (ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger).
tensorflow.keras.preprocessing.image: Để tiền xử lý và tăng cường dữ liệu ảnh (ImageDataGenerator, load_img, img_to_array).
warnings: Để quản lý cảnh báo.
pandas (as pd): Dù được import trong một khối code Kaggle boilerplate, nó không được sử dụng trong luồng chính của mô hình bạn cung cấp.
io: Sử dụng để xử lý file ảnh tải lên trong bộ nhớ (BytesIO).
2. Kiến trúc và Mô hình được sử dụng:

Mô hình được xây dựng là một mạng nơ-ron tích chập (Convolutional Neural Network - CNN) tùy chỉnh, được định nghĩa trong hàm build_cnn_model().

Kiến trúc chi tiết của mô hình CNN này bao gồm:

Ba khối tích chập (Convolutional Blocks): Mỗi khối gồm:
Conv2D (lớp tích chập 2D) với các kích thước bộ lọc (filter) tăng dần (32, 64, 128).
BatchNormalization: Chuẩn hóa đầu ra của lớp tích chập để ổn định quá trình huấn luyện.
MaxPooling2D: Giảm kích thước không gian của bản đồ đặc trưng.
Dropout: Ngẫu nhiên bỏ qua một số kết nối để giảm overfitting (với tỷ lệ 0.25, 0.25, 0.30 cho mỗi khối).
Phần phân loại (Classifier Head):
Flatten: Chuyển đổi đầu ra từ các khối tích chập thành một vector 1D.
Dense (lớp kết nối đầy đủ) với 256 nơ-ron, theo sau là BatchNormalization và Dropout (tỷ lệ 0.40).
Dense (lớp đầu ra) với số nơ-ron bằng số lượng lớp cảm xúc (NUM_CLASSES, tức 7), sử dụng hàm kích hoạt softmax để đưa ra xác suất cho từng lớp.
Hàm kích hoạt: relu được sử dụng cho các lớp ẩn, và softmax cho lớp đầu ra.
Kernel Initializer: he_normal được sử dụng cho các lớp Conv2D.
Mô hình Deep Learning Cụ thể:

Là một mô hình Convolutional Neural Network (CNN) được xây dựng tùy chỉnh bằng Keras Sequential API, được thiết kế đặc biệt để phân loại 7 loại cảm xúc từ ảnh khuôn mặt thang độ xám (48x48 pixel).
Mặc dù các mô hình như SVM, KNN, LBP (truyền thống) hay ResNet, Inception (deep learning tiên tiến hơn), OpenCV, và DeepFace (thư viện hỗ trợ) được nhắc đến trong yêu cầu của bạn, chúng không được sử dụng hoặc triển khai trực tiếp trong các khối code hiện có của notebook này. Notebook tập trung vào việc xây dựng và huấn luyện một mô hình CNN từ đầu sử dụng TensorFlow/Keras.